In [1]:
from gwcs import coordinate_frames as cf, wcs
from astropy import coordinates as coord
from astropy import units as u
from astropy.time import Time
from astropy.modeling import models
from astropy.tests.helper import assert_quantity_allclose

from numpy.testing import assert_allclose

In [2]:
slit = cf.CoordinateFrame(naxes=1, axes_type='SPATIAL', axes_order=(0,),
                          axes_names=('slit',), unit=u.pix)

spec=cf.SpectralFrame(name='wave', unit=[u.m,], axes_order=(0,),
                      axes_names=('lambda',))


In [3]:
# correctly errors
cf.CompositeFrame([slit, spec])

# axes_order is used to define the oredring of the rest of the attributes


ValueError: Incorrect numbering of axes, axes_order should contain unique numbers, got [0, 0].


Current issues

- When Celestial axes are swapped, CelestialFrame.coordinates() does not work - **fixed now**

- Mixed cooridnates ra, lam, dec - **fixed now**

In [4]:
spec1=cf.SpectralFrame(name='wave', unit=[u.m,], axes_order=(2,),
                      axes_names=('lambda',))
cel1=cf.CelestialFrame(reference_frame=coord.ICRS(), axes_order=(1,0))
comp=cf.CompositeFrame([spec1, cel1]) # this should be fixed --> inconsistency 
# between frames order and axes_order, perhaps just raise an error?

print(comp.coordinates(2,3,4))
print(comp.axes_type)
# Note that the order of the frames does not matter, the ordering of axes_order does

[<Quantity 4. m>, <SkyCoord (ICRS): (ra, dec) in deg
    (3., 2.)>]
('SPATIAL', 'SPATIAL', 'SPECTRAL')


In [5]:
cel2 = cf.CelestialFrame(reference_frame=coord.ICRS(), axes_order=(2, 0))
spec2=cf.SpectralFrame(name='wave', unit=[u.m,], axes_order=(1,),
                      axes_names=('lambda',))
comp=cf.CompositeFrame([cel2, spec2])
comp.coordinates(2,3,4)

[<SkyCoord (ICRS): (ra, dec) in deg
     (4., 2.)>,
 <Quantity 3. m>]

In [6]:
comp.axes_type

('SPATIAL', 'SPECTRAL', 'SPATIAL')

In [7]:
model_2d_shift = models.Shift(1) & models.Shift(2)
out_frame = cf.CelestialFrame(reference_frame=coord.ICRS(), axes_order=(1, 0))
w = wcs.WCS(model_2d_shift, output_frame=out_frame)
assert_allclose(model_2d_shift(3, 3), (4.0, 5.0))
result = w(3, 3, with_units=True)
result
#assert result == coord.SkyCoord(5., 4., unit=(u.deg, u.deg))

<SkyCoord (ICRS): (ra, dec) in deg
    (5., 4.)>

In [8]:
icrs = cf.CelestialFrame(reference_frame=coord.ICRS(), axes_order=(2, 0))
spec = cf.SpectralFrame(name='wave', unit=[u.m,], axes_order=(1,), axes_names=('lambda',))
output_frame = cf.CompositeFrame(frames=[icrs, spec])
assert output_frame.axes_order == (2, 0, 1)
assert output_frame.axes_names == ('lat', 'lambda', 'lon')
w = wcs.WCS(forward_transform=models.Identity(3), output_frame=output_frame)
assert w(1,2,3) == (1,2 ,3)
w(1,2,3, with_units=True)

[<SkyCoord (ICRS): (ra, dec) in deg
     (3., 1.)>,
 <Quantity 2. m>]

In [9]:
slit = cf.CoordinateFrame(naxes=1, axes_type='SPATIAL', axes_order=(0,),
                          axes_names=('slit',), unit=u.pix)

spec=cf.SpectralFrame(name='wave', unit=[u.m,], axes_order=(1,),
                      axes_names=('lambda',))
output_frame = cf.CompositeFrame(frames=[slit, spec])
w = wcs.WCS(forward_transform=models.Identity(2), output_frame=output_frame)
result = w(1, 2, with_units=True)
assert_quantity_allclose(result[0], 1*u.pix)
assert_quantity_allclose(result[1], 2*u.m)

In [10]:
slit = cf.CoordinateFrame(naxes=1, axes_type='SPATIAL', axes_order=(1,),
                          axes_names=('slit',), unit=u.pix)

spec=cf.SpectralFrame(name='wave', unit=[u.m,], axes_order=(0,),
                      axes_names=('lambda',))
output_frame = cf.CompositeFrame(frames=[spec, slit])
w = wcs.WCS(forward_transform=models.Identity(2), output_frame=output_frame)
result = w(1, 2, with_units=True)
result
#assert_quantity_allclose(result[0], 1*u.m)
#assert_quantity_allclose(result[1], 2*u.m)

[<Quantity 1. m>, (<Quantity 2. pix>,)]

In [11]:
cel=cf.CelestialFrame(reference_frame=coord.ICRS(), axes_order=(0,1))

In [ ]:

def reorder_axes(self, coofr, axes_order):
    # again problem is list of frames
    self.insert_transform(Mapping(axes_order), before=coofr)
    self.coofr

    